In [ ]:
!playwright install

In [ ]:
import requests
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import regex
import os

In [ ]:
url = "https://www.nyc.gov/site/manhattancb1/archives/committee-meeting-agendas.page"

In [ ]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = True)
page = await browser.new_page()

await page.goto(url)

In [ ]:
html = await page.content()
doc = BeautifulSoup(html)

In [ ]:
tabs = doc.find(class_="agencies-about-links").find_all('li')

In [ ]:
tab_links = []

for tab in tabs:
  link = tab.a
  tab_links.append(link['href'])

tab_links

In [ ]:
root_url = 'https://www.nyc.gov'

In [ ]:
def create_folder(category):
    folder_path = os.path.join('/content/pdfs', category)
    os.makedirs(folder_path, exist_ok=True)

In [ ]:
for link in tab_links:
  full_url = root_url + link
  last_part = full_url.split('/')[-1]
  category = last_part.replace('.page', '')
  create_folder(category)

  # Then scrape all the links for each page and save them in the right folder.
  await page.goto(full_url)
  html = await page.content()
  doc = BeautifulSoup(html)
  files = doc.find(class_="about-main-image").find_all('a')
  for file in files:
    file_url = file['href']
    file_name = file_url.split('/')[-1]
    file_name = file_name.replace(' ', '_')
    response = requests.get(root_url + file_url)
    with open(f"/content/pdfs/{category}/{file_name}", 'wb') as f:
        f.write(response.content)
  print(f"Got files for {category}")

In [ ]:
!cp -r /Users/thomaslee/Desktop/G1-2/Studio/CommunityBoard

In [ ]:
!ls /Users/thomaslee/Desktop/G1-2/Studio/CommunityBoard